In [1]:
import zstandard as zstd
import io
import json
from nnsight import LanguageModel

/home/can/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map='cuda:0')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# set up data as a generator
data_path = '/share/data/datasets/pile/the-eye.eu/public/AI/pile/train/00.jsonl.zst'
compressed_file = open(data_path, 'rb')
dctx = zstd.ZstdDecompressor()
reader = dctx.stream_reader(compressed_file)
text_stream = io.TextIOWrapper(reader, encoding='utf-8')

def generator():
    for line in text_stream:
        yield json.loads(line)['text']
data = generator()

def text_batch(batch_size):
        """
        Return a list of text
        """
        return [
            next(data) for _ in range(batch_size)
        ]

In [24]:
len_tokenized_prompt = 50

prompt_str = next(data)
prompt_tokid = model.tokenizer.encode(prompt_str, return_tensors='pt')[:len_tokenized_prompt]

print(f'shape of prompt_tokid: {prompt_tokid.shape}')
print(f'prompt:\n{model.tokenizer.decode(prompt_tokid[0])}')

shape of prompt_tokid: torch.Size([1, 159])
prompt:
Running

Stat

Dinner with people is always better than eating alone, especially when the food is good. Good food tastes even better when enjoyed with people. Tonight Amy came over to try my second attempt at the Brussels Sprouts Veggie Soup to which I have made some changes (see recipe below in previous post) for a better result, I believe.

We were at the store earlier and saw some nice looking haricot verts and heirloom tomatoes, so we decide to assemble a simple salad from those. Of course while I’m at the market, I can’t not get some five peppercorn salami. Our simple dinner of soup, salami, bread, cheese, salad, and wine was on the table in 15 minutes.


In [ ]:
# Cache the gradients of the correct logit w.r.t dictionary features